# 变量作用域
变量的作用域是代码的一个区域，在这个区域中这个变量是可见的。给变量划分作用域有助于解决变量命名冲突。这个概念是符合直觉的：两个函数可能同时都有叫做x的参量，而这两个x并不指向同一个东西。相似地，也有很多其他的情况下代码的不同块会使用同样名字而并不指向同一个东西。相同的变量名是否指向同一个东西的规则被称为作用域规则；这一节会详细地把这个规则讲清楚。

语言中的某个创建会引入作用域块，这是代码中的一个区域，有资格成为一些变量集合的作用域。一个变量的作用域不可能是源代码行的任意集合；相反，它始终与这些块之一关系密切。在Julia中有两个主要类型的作用域，全局作用域与局部作用域，后者可以嵌套。引入作用域块的创建是：

* 只能在其他全局作用域块中嵌套的作用域块：

  * 全局作用域

    * 模块，裸模块

    * 在交互式提示行（REPL）

  * 局部作用域（不允许嵌套）

    * （可变的）结构，宏
* 可以在任何地方嵌套的作用域块（在全局或者局部作用域中）：

  * 局部作用域

    * for，while，try-catch-finally，let

    * 函数（语法，匿名或者do语法块）

    * 推导式，broadcast-fusing

值得注意的是，**这个表内没有的是 begin 块和 if 块，这两个块不会引进新的作用域块**。这两种作用域遵循的规则有点不一样。
### 词语法作用域
Julia使用词法作用域，也就是说一个函数的作用域不会从其调用者的作用域继承，而从函数定义处的作用域继承。
```Julia
module Bar
x=1
foo()=x
end;
import .Bar
x=-1
Bar.foo()#返回1
```
## 全局作用域
每个模块会引进一个新的全局作用域，与其他所有模块的全局作用域分开；无所不包的全局作用域不存在。模块可以把其他模块的变量引入到它的作用域中，通过using 或者 import语句或者通过点符号这种有资格的通路，也就是说**每个模块都是所谓的命名空间**。值得注意的是变量绑定只能在它们的全局作用域中改变，在外部模块中不行。<br/>
```Julia
module A
a=1
end;
module B
module C
c=2
end
b=C.c
import ..A
d=A.a
end;
module D
b=a#error
end;
module E
import ..A
A.a=2#error
end;
```
**注意交互式提示行（即REPL）是在模块Main的全局作用域中。**<br/>
## 局部作用域
大多数代码块都会引进一个新的局部作用域（参见上面的表以获取完整列表）。局部作用域会从父作用域中继承所有的变量，读和写都一样。另外，局部作用域还会继承赋值给其父全局作用域块的所有全局变量（如果由全局if或者begin作用域包围）。不像全局作用域，**局部作用域并不是命名空间**，所以在其内部作用域中的变量无法通过一些合格的通路在其父作用域中得到。

接下来的规则和例子都适用于局部作用域。 在局部作用域中新引进的变量不会反向传播到其父作用域。for，while，try-catch-finally，let等<br/>

**在局部作用域中可以使用local关键字来使一个变量强制为新的局部变量。**

**在局部作用域内部，可以使用global关键字赋值给一个全局变量**

**在作用域块中local和global关键字的位置都无关痛痒。**

local和global关键字都可以用于解构赋值，也就是说local x, y = 1, 2。在这个例子中关键字影响所有的列出来的变量。

大多数块关键字都会引入局部作用域，而begin和if是例外。

在一个局部作用域中，所有的变量都会从其父作用域块中继承，除非：

赋值会导致全局变量改变，或者
变量专门使用local关键字标记。
所以全局变量只能通过读来继承，而不能通过写来继承。
```Julia
x,y=1,2
function foo()
    x=2
    return x+y
end
foo()#4
x#1
```
***应谨慎使用全局变量，避免改变全局变量的值***
```Julia
function foobar()
    global x=2
end
foobar()
x#return 2
```
**嵌套函数会改变其父作用域的局部变量**
```Julia
x,y=1,2;
function baz()
    x=2#创建新的局部变量
    function bar()
        x=10#修改父变量
        return x+y#y是局部变量
    end
    return bar()+x#12+10#x修改并调用bar()
end
baz()#22
x,y#(1,2)
```
** 允许嵌套函数修改其父作用域的局部变量的原因是允许构建闭包， 闭包中有一个私有的态**，例如下面例子中的state变量：

```Julia
let state=0
global counter()=(state+=1)
end
counter()
counter()
```
例如在第一个例子中的x与在第二个例子中的state，**内部函数从包含它的作用域中继承的变量有时被称为捕获变量**。捕获变量会带来性能挑战，这会在性能建议中讨论。继承全局作用域与嵌套局部作用域的区别会导致在局部或者全局作用域中定义的函数在变量赋值上的稍许区别。

```Julia
x,y=1,2;
function bar()
    x=10#local,no longer a closure variable
    return x+y
end
function quz()
    x=2#local
    return bar()+x#12+2
```
注意到在上面的嵌套规则并不适用于类型和宏定义因为他们只能出现在全局作用域中。涉及到函数中提到的默认和关键字函数参数的评估的话会有特别的作用域规则。<br/>
**在函数，类型或者宏定义内部使用的变量，将其引入到作用域中的赋值行为不必在其内部使用之前进行**<br/>
```Julia
f=y->y+a
f(3)#报错
a=1
f(3)#4
```
这个行为看起来对于普通变量来说有点奇怪，但是这个允许命名过的函数 – 它只是连接了函数对象的普通变量 – 在定义之前就能被使用。这就允许函数能以符合直觉和方便的顺序定义，而非强制以颠倒顺序或者需要前置声明，只要在实际调用之前被定义就行。

## let块
，let语句每次运行都新建一个新的变量绑定。赋值改变的是已存在值的位置，let会新建新的位置。这个区别通常都不重要，只会在通过闭包跳出作用域的变量的情况下能探测到。let语法接受由逗号隔开的一系列的赋值和变量名：

## 循环和推导式
for循环，while循环，和Comprehensions拥有下述的行为：任何在它们的内部的作用域中引入的新变量在每次循环迭代中都会被新分配一块内存，就像循环体是被let块包围一样。<br/>
for循环或者推导式的迭代变量始终是个新的变量<br/>
但是，有时重复使用一个存在的变量作为迭代变量是有用的。 这能够通过添加关键字**outer**来方便地做到：<br/>
const声明只应该在全局作用域中对全局变量使用。编译器很难为包含全局变量的代码优化，因为它们的值（甚至它们的类型）可以任何时候改变。如果一个全局变量不会改变，添加const声明会解决这个问题。<br/>
const只会影响变量绑定；变量可能会绑定到一个可变的对象上（比如一个数组）使得其任然能被改变。另外当尝试给一个声明为常量的变量赋值时下列情景是可能的：**如果一个新值的类型与常量类型不一样时会扔出一个错误,如果一个新值的类型与常量一样会打印一个警告,如果赋值不会导致变量值的变化，不会给出任何信息;最后一条规则适用于不可变对象，即使变量绑定会改变，** 

        

In [ ]:
even(n)=(n==0) ? true : odd(n-1);
odd(n)=(n==0) ? true : even(n-1);

even(3)
odd(3)
#Julia提供了叫做iseven和isodd的内置的高效的奇偶性检验的函数
x,y,z=-1,-1,-1;
let x=1;z=-z
    println("x:$x,y:$y")
    println("z:$z")
end
x,y,z
#在左边的新变量被引入之前右边的每隔两都会在作用域中被评估。
#let x = x这样的东西是合法的，因为两个x变量是不一样的，拥有不同的存储位置。
#但会全局地改变x的值

#####################
Fs=Vector{Any}(undef,2);i=1
while i<=2
    Fs[i]=()->i#必须如此定义？
    global i+=1
end
println(Fs[1]())
println(Fs[2]())

######################
Fs=Vector{Any}(undef,2);i=1;
while i<=2
    let i=i#必须为变量
        Fs[i]=()->i 
    end
    global i+=1
end
println(Fs[1]())
println(Fs[2]())

################### 
#因为 begin 结构不会引入新的作用域，
#使用没有参数的 let 来只引进一个新的作用域块而不创建新的绑定
let 
    local x=1
    let
        local x=2#let 和 end内的local只作用于其内
    end
    x
end

#因为let引进了一个新的作用域块，内部的局部x与外部的局部x是不同的变量。

###循环和推导式
Fs=Vector{Any}(under,2);
for j=1:2
    Fs[j]=()->j
end
Fs[1]()#1
Fs[2]()#1

#for循环或者推导式的迭代变量始终是个新的变量
function f()
    i=0
    for i=1:3
    end
    return i
end
f()#0

#但是，有时重复使用一个存在的变量作为迭代变量是有用的。 
#这能够通过添加关键字outer来方便地做到
function f()
    i=0
    for outer i=1:3#添加outer关键字将i设为外部迭代变量
    end
    return i
end
f()#3


########################常量
const e,pi=2.71828182845904523526,3.14159265358979323846

#如果赋值不会导致变量值的变化，不会给出任何信息
#最后一条规则适用于不可变对象，即使变量绑定会改变，绑定=地址
const s1="1"
s2="1"
pointer.([s1,s2],1)#地址不同
s1=s2
pointer.([s1,s2],1)#地址相同

#但是对于可变对象，警告会如预期出现
const a=[1]
a=[1]#warning

#即使可能，改变一个声明为常量的变量的值是十分不推荐的。
#例如，如果一个方法引用了一个常量并且在常量被改变之前已经被编译了，
#那么这个变量还是会保留使用原来的值：

const x=1
f()=x
f()
x=2
f()#1